In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

KeyboardInterrupt: 

# 데이터 불러오기

In [ ]:
train = pd.read_csv(r'C:\Users\yoon9\PycharmProjects\Sejong_Project\AI Class\Data\real-estate\train(5).csv')
test = pd.read_csv(r'C:\Users\yoon9\PycharmProjects\Sejong_Project\AI Class\Data\real-estate\test(5).csv')

In [ ]:
train.shape, test.shape

In [ ]:
y = train['SalePrice']
df = pd.concat([train,test], axis = 0)
df = df.drop(columns=['SalePrice'])

In [ ]:
df.shape

# 데이터 결측치 처리

수치형 변수

In [ ]:
df.isnull().sum()


In [ ]:
df = df.drop(columns=['Alley','PoolQC','Fence','MiscFeature'])

# 우선 총 데이터 3000개 중에서 2/3 이상이 결측치인 변수는 제거해주도록 하겠습니다.

# 변수 줄이기

변수가 너무 많기 때문에 다중공산성을 확인하여 변수를 제거할 계획이다.

In [ ]:
# 상관행렬 시각화

plt.figure(figsize=(20,20))
sns.heatmap(df.corr(), annot = True, cmap = 'RdYlGn_r')
plt.show()

In [ ]:
df.corr() > 0.7

df = df.drop(columns = ['YearBuilt','1stFlrSF','GrLivArea','GarageCars'])

# 4가지의 상관계수가 높은 변수들 제거

# 데이터 타입별 변수 구분

In [ ]:
df_type = pd.DataFrame(df.dtypes)

# 각 데이터 타입별로 전처리 진행

In [ ]:
df_int =  df_type[(df_type[0] == 'int64') | (df_type[0] == 'float')].index

# int , float 타입 변수

In [ ]:
df_int.dtype

In [ ]:
df_object = df_type[(df_type[0] == 'object')].index

# 문자형 변수들

## 결측치 filling

변수는 특정 값이 영향을 줄 수 있기 때문에 최빈값으로 채워넣겠습니다.

In [ ]:
df[df_int].isnull().sum()

In [ ]:
df = df.fillna(df.mode().loc[0])

# 최빈값으로 채워넣기

In [ ]:
df.isnull().sum()

# 결측치 제거 확인

## 피처 엔지니어링

In [ ]:
from sklearn.preprocessing import StandardScaler # 스케일링 라이브러리
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
num_attribs = df[df_int].drop(columns=['MSSubClass','OverallQual','OverallCond'])
cat_1hot_attribs = df[df_object]
cat_ord_attribs = df[['MSSubClass','OverallQual','OverallCond']]
# 변수별 타입 나누기

# 스케일링

In [ ]:
# 수치형 변수 스케일링
scaler = StandardScaler()
X_scaler  = scaler.fit(num_attribs)
X_scale = X_scaler.transform(num_attribs)
X_scale = pd.DataFrame(X_scale, columns=num_attribs.columns)
X_scale  # 스케일링 데이터 프레임화

In [ ]:
X_scale['Id'] = df['Id'].values


# 인코딩

In [ ]:
# 원핫인코딩 함수 적용
hot1_encoder = OneHotEncoder(sparse=False)
X_1hot = hot1_encoder.fit_transform(cat_1hot_attribs)
X_1hot

In [ ]:
X_1hot = pd.DataFrame(X_1hot, columns=hot1_encoder.get_feature_names()) #순서가 없는 카테고리형
# 데이터 프레임화

In [ ]:
X_1hot

In [ ]:
ord_encoder = OrdinalEncoder() #순서가 있는 카테고리형
X_ord = ord_encoder.fit_transform(cat_ord_attribs)
X_ord = pd.DataFrame(X_ord, columns= ['MSSubClass','OverallQual','OverallCond'])
X_ord

In [ ]:
df_precessed = pd.concat([X_scale, X_1hot,X_ord], axis = 1)
df_precessed
#최종 데이터 합치기

In [ ]:
df_precessed

# 데이터 나누기

In [ ]:
X = df_precessed[:1460]
test = df_precessed[1460:]

# train, test 나누기
X.shape, test.shape, y.shape

# train, val 데이터셋 나누기

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size= 0.3, shuffle= 1004)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

# 모델 학습

In [ ]:
from sklearn.ensemble import RandomForestRegressor
#랜덤포레스트 사용

model = RandomForestRegressor()

In [ ]:
model.fit(X_train,y_train)

In [ ]:
X_predicted = model.predict(X_val)

# 모델 평가

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [ ]:
print(f'r2 score : {r2_score(y_val, X_predicted)}')
print(f'RMSE : {np.sqrt(mean_squared_error(y_val, X_predicted))}')
print(f'MAE : {mean_absolute_error(y_val, X_predicted)}')

# 변수 중요도 확인

In [ ]:
plt.figure(figsize= (15,12))  # 피쳐의 중요도, 가중치 시각화
sns.barplot(x = X_train.columns, y =model.feature_importances_)
plt.xticks(rotation = 90)
plt.show()

In [ ]:
#model.feature_importances_
df_import =  pd.DataFrame(index= X_train.columns, columns = ['변수 중요도'],data = model.feature_importances_)

In [ ]:
importance_features = df_import[df_import['변수 중요도'] > 0.001].index

In [ ]:
df_selected = df_precessed[importance_features]

# 변수 선택 후 모델 학습

# 데이터 나누기

In [ ]:
X_selected = df_selected[:1460]
test_selected = df_selected[1460:]

# train, test 나누기
X_selected.shape, test_selected.shape, y.shape

# train, val 데이터셋 나누기

In [ ]:
from sklearn.model_selection import train_test_split

X_s_train, X_s_val, y_s_train, y_s_val = train_test_split(X_selected, y, test_size= 0.3, shuffle= 1004)
X_s_train.shape, X_s_val.shape, y_s_train.shape, y_s_val.shape

# 모델 학습

In [ ]:
from sklearn.ensemble import RandomForestRegressor
#랜덤포레스트 사용

model_s = RandomForestRegressor()

In [ ]:
model_s.fit(X_s_train,y_s_train)

In [ ]:
X_s_predicted = model_s.predict(X_s_val)

# 모델 평가

In [ ]:
print(f'r2 score : {r2_score(y_s_val, X_s_predicted)}')
print(f'RMSE : {np.sqrt(mean_squared_error(y_s_val, X_s_predicted))}')
print(f'MAE : {mean_absolute_error(y_s_val, X_s_predicted)}')

In [ ]:
plt.figure(figsize= (15,12))  # 피쳐의 중요도, 가중치 시각화
sns.barplot(x = X_s_train.columns, y =model_s.feature_importances_)
plt.xticks(rotation = 90)
plt.show()

# 파라미터 튜닝

In [ ]:

!pip install --user optuna

In [ ]:
from sklearn.metrics import accuracy_score
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import cross_val_score

In [ ]:
def RMSE(y, y_pred):
    rmse = mean_squared_error(y, y_pred) ** 0.5
    return rmse

In [ ]:
def RF_objective(trial):
    max_depth = trial.suggest_int('max_depth', 1, 10)
    max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 2, 1000)
    n_estimators =  trial.suggest_int('n_estimators', 100, 500)

    model_o = RandomForestRegressor(max_depth = max_depth, max_leaf_nodes = max_leaf_nodes,n_estimators = n_estimators,n_jobs=2,random_state=25)


    model_o.fit(X_train, y_train)

    rmse = RMSE(y_val, model_o.predict(X_val))
    return rmse

#Execute optuna and set hyperparameters
RF_study = optuna.create_study(direction='minimize')
RF_study.optimize(RF_objective, n_trials=30)

#Create an instance with tuned hyperparameters
optimized_RF = RandomForestRegressor(max_depth = RF_study.best_params['max_depth'], max_leaf_nodes = RF_study.best_params['max_leaf_nodes'],
                                      n_estimators = RF_study.best_params['n_estimators'],n_jobs=2,random_state=25)

In [ ]:
optimized_RF.fit(X_train,y_train)

In [ ]:
opt_pred = optimized_RF.predict(X_val)

In [ ]:
print(f'RMSE : {np.sqrt(mean_squared_error(y_val, opt_pred))}')
print(f'MAE : {mean_absolute_error(y_val, opt_pred)}')

# submission

In [ ]:
test_predicted = optimized_RF.predict(test)

In [ ]:
submission = pd.read_csv(r'C:\Users\yoon9\PycharmProjects\Sejong_Project\AI Class\Data\real-estate\submission(5).csv')
submission

In [ ]:
submission['SalePrice'] = test_predicted

In [ ]:
submission.to_csv(r'C:\Users\yoon9\PycharmProjects\Sejong_Project\AI Class\Data\real-estate\submission_17011733_윤지환.csv',index = False)